<a href="https://colab.research.google.com/github/sanjayathreya/CHET/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/sanjayathreya/CHET.git

Cloning into 'CHET'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), 2.85 MiB | 6.17 MiB/s, done.


In [2]:
!pip install pyhealth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.6 MB/s eta 0:00:00


In [7]:
%cd /content/CHET

/content/CHET


In [8]:
%pwd

'/content/CHET'

In [35]:
import pandas as pd
from pyhealth.medcode import CrossMap
from pyhealth.datasets import MIMIC4Dataset,MIMIC3Dataset
from pyhealth.medcode import InnerMap
icd9cm = InnerMap.load("ICD9CM")

In [11]:
mimic3_ds = MIMIC3Dataset(
    # Argument 1: It specifies the data folder root.
    root="data/mimic3/raw",

    # Argument 2: The users need to input a list of raw table names (e.g., DIAGNOSES_ICD.csv, PROCEDURES_ICD.csv).
    tables=["DIAGNOSES_ICD"],

    # Argument 3: This argument input a dictionary (key is the source code
    # vocabulary and value is the target code vocabulary .
    # Default is empty dict, which means the original code will be used.
)

Mapping codes: 100%|██████████| 23692/23692 [00:00<00:00, 68211.72it/s]


In [37]:
from collections import OrderedDict
patient_admission = OrderedDict()
admission_codes = OrderedDict()

patient_dict = mimic3_ds.patients
print(list(patient_dict.keys())[:10])
patient = patient_dict["1004"]
visit_dict = patient.visits
len(visit_dict)
admissions =[]
for visit_key, visit_values in visit_dict.items():
  admissions.append({'adm_id': visit_key, 'adm_time': visit_values.encounter_time}) 
  diagnoses = visit_values.get_code_list(table="DIAGNOSES_ICD")
  diagnoses_std = [icd9cm.standardize(code) for code in diagnoses]
  admission_codes[visit_key] = diagnoses_std

patient_admission[patient] = sorted(admissions, key=lambda admission: admission['adm_time'])

print(patient_admission)
print(admission_codes)


['10', '100', '1000', '10000', '10001', '10002', '10005', '10006', '10007', '10008']
OrderedDict([(Patient 1004 with 2 visits, [{'adm_id': '127794', 'adm_time': datetime.datetime(2108, 5, 10, 8, 0)}, {'adm_id': '177186', 'adm_time': datetime.datetime(2108, 6, 7, 21, 52)}])])
OrderedDict([('127794', ['437.3', '518.5', '997.2', '444.22', '997.02', '588.1', '599.0', 'E879.8', '998.2', '403.90', '753.12', '296.80', '378.51', 'V58.65', '443.0']), ('177186', ['434.91', '444.22', '276.7', '287.4', '584.9', '998.11', '276.0', '276.2', '452', '573.4', '570', '588.1', 'E934.2', '401.9', '753.12', '296.7', '272.4'])])
